### Projet ETL

Le projet est composé de deux fichiers 'usertable.csv' et 'VisitorLogs.csv'. L'objectif de cette base de données d'en faire une base de donnée pour qu'une équipe de marketing puisse l'utiliser.


## Implémentation bibliothèque


In [78]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message=".*SettingWithCopyWarning.*")

## Extract

La table de 'userTable.csv' 

In [79]:
df_user=pd.read_csv('userTable.csv',dtype=str)
df_user.head()

,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


La table de 'VIsitorLogsData.csv'

In [80]:
df_VisitorLogs=pd.read_csv('VisitorLogsData.csv')
df_VisitorLogs.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


## Cleaning des données
Il y'a plusieurs à transformer dans nos données.  
Dans 'VisitorLogsData.csv'. On travaille uniquement avec les données des membres inscrits, on supprime  donc toutes les lignes qui ne possède pas de UserID.  
De plus, on formate les dates aussi, on garde uniquement les lignes avec des dates et on garde le format 'YYYY-MM-DD ... '    

In [81]:
df_VisitorLogs=df_VisitorLogs.dropna(subset=['VisitDateTime'])
df_VisitorLogs=df_VisitorLogs.dropna(subset=['UserID'])

df_VisitorLogs['VisitDateTime']=pd.to_datetime(df_VisitorLogs['VisitDateTime'],errors='coerce',unit='ns')
df_VisitorLogs = df_VisitorLogs[df_VisitorLogs['VisitDateTime'].notna()]
df_VisitorLogs.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
23,WI100000223826,2018-05-10 06:28:53.391,Pr100241,U132443,NaN,Firefox,Windows,NaN,India
24,WI10000021998,2018-05-08 12:40:02.153,pr100495,U134616,click,Chrome,Windows,Cottage Grove,United States
33,WI10000036281,2018-05-11 15:35:43.689,Pr100363,U130784,click,Chrome,Chrome OS,New Delhi,India
50,WI100000110130,2018-05-19 00:02:31.347,pr100340,U120983,click,Chrome,Windows,Mumbai,India
54,WI10000014616,2018-05-19 04:51:45.337,Pr100166,U120287,click,Chrome,Windows,NaN,NaN


## Nouveau DataFrame  
On crée un noueau DataFrame final avec les différents données que l'on veut voir.

In [82]:
colonnes = ['UserID','No_of_days_Visited_7_Days','No_Of_Products_Viewed_15_Days', 'User_Vintage','Most_Viewed_product_15_Days',
            'Most_Active_OS','Recently_Viewed_Product','Pageloads_last_7_days','Clicks_last_7_days']

df_Marketing_Model=pd.DataFrame(columns=colonnes)
df_Marketing_Model['UserID']= df_VisitorLogs['UserID'].unique()
df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U134616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U130784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,U120983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U120287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Travail sur les 7 derniers jours 
On va travailler avec des données qui sont sur les 7  ou 15 derniers jours. 

In [83]:
date_fin = pd.Timestamp('2018-05-27')
date_debut = date_fin - pd.Timedelta(days=7)
df_7jours= df_VisitorLogs[(df_VisitorLogs['VisitDateTime'] >= date_debut) & (df_VisitorLogs['VisitDateTime'] <= date_fin)]
df_7jours.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
68,WI10000018039,2018-05-23 09:44:44.023,Pr101042,U113937,CLICK,Safari,mac os x,Dailekh,Nepal
106,WI100000319667,2018-05-25 06:01:57.465,pr100044,U111160,click,Chrome Mobile,Android,Bengaluru,India
140,WI1000001500,2018-05-21 05:28:46.221,Pr100339,U119431,click,Chrome,windows,Kolkata,India
198,WI100000506560,2018-05-25 17:03:00.226,Pr100503,U109651,PAGELOAD,Chrome,windows,Bengaluru,India
213,WI100000727623,2018-05-21 03:55:35.958,pr100544,U113103,pageload,Chrome,Windows,Chennai,India


In [84]:
date_fin = pd.Timestamp('2018-05-27')
date_debut = date_fin - pd.Timedelta(days=15)
df_15jours= df_VisitorLogs[(df_VisitorLogs['VisitDateTime'] >= date_debut) & (df_VisitorLogs['VisitDateTime'] <= date_fin)]
df_15jours.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
50,WI100000110130,2018-05-19 00:02:31.347,pr100340,U120983,click,Chrome,Windows,Mumbai,India
54,WI10000014616,2018-05-19 04:51:45.337,Pr100166,U120287,click,Chrome,Windows,NaN,NaN
68,WI10000018039,2018-05-23 09:44:44.023,Pr101042,U113937,CLICK,Safari,mac os x,Dailekh,Nepal
69,WI1000001200,2018-05-13 13:17:03.751,Pr101042,U115735,click,Chrome,Windows,Khammam,India
74,WI10000022937,2018-05-16 12:39:44.279,Pr101964,U103943,click,Chrome,Windows,NaN,NaN


# Nombres de visites des utilisateurs sur 7 Jours
On veut regarder combien de visiteurs a-t-on eu les 7 derniers jours pour possiblement voir l'évolution des visites du site.

In [85]:
# Compter les visites par utilisateur
visit_counts = df_7jours['UserID'].value_counts().reset_index()
visit_counts.columns = ['UserID', 'No_of_days_Visited_7_Days']
# Effectuer la jointure avec les nouvelles valeurs
updated_values = df_Marketing_Model[['UserID']].merge(visit_counts, on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model['No_of_days_Visited_7_Days']=df_Marketing_Model['No_of_days_Visited_7_Days'].fillna(0)
pd.set_option('future.no_silent_downcasting', True)
df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U134616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U130784,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,U120983,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U120287,143.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 # Nombres de produits vues par les utilisateurs sur les derniers 15 Jours
 

In [86]:
nbProduit15Days= df_15jours.groupby('UserID')['ProductID'].count().reset_index()
nbProduit15Days.columns=['UserID','No_Of_Products_Viewed_15_Days']
updated_values = df_Marketing_Model[['UserID']].merge(nbProduit15Days, on='UserID', how='left')
df_Marketing_Model.update(updated_values)

df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U134616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U130784,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,U120983,1.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN
4,U120287,143.0,485.0,NaN,NaN,NaN,NaN,NaN,NaN


# Durée du compte

In [87]:
User_Vintage= df_user[['UserID','Signup Date']]
User_Vintage.loc[:, 'Signup Date'] = pd.Timestamp('2018-05-27', tz='UTC') - pd.to_datetime(User_Vintage['Signup Date'], format='ISO8601')
User_Vintage.columns=['UserID','User_Vintage']
# print(User_Vintage['Signup Date'].head())
updated_values = df_Marketing_Model[['UserID']].merge(User_Vintage, on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,239 days 05:33:17.252592,NaN,NaN,NaN,NaN,NaN
1,U134616,0,NaN,446 days 08:55:01.624228,NaN,NaN,NaN,NaN,NaN
2,U130784,0,NaN,261 days 05:15:35.831008,NaN,NaN,NaN,NaN,NaN
3,U120983,1.0,32.0,76 days 18:53:27.568512,NaN,NaN,NaN,NaN,NaN
4,U120287,143.0,485.0,45 days 05:37:40.435553,NaN,NaN,NaN,NaN,NaN


# Produit le plus observé les 15 derniers jours par utilisateur

In [88]:
produit_counts = df_15jours.groupby(['UserID', 'ProductID']).size().reset_index(name='count')
produit_max = produit_counts.loc[produit_counts.groupby('UserID')['count'].idxmax()]
produit_max.columns=['UserID','Most_Viewed_product_15_Days','count']
updated_values = df_Marketing_Model[['UserID']].merge(produit_max[['UserID', 'Most_Viewed_product_15_Days']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model['Most_Viewed_product_15_Days']=df_Marketing_Model['Most_Viewed_product_15_Days'].fillna('Product101')
df_Marketing_Model.head()


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,239 days 05:33:17.252592,Product101,NaN,NaN,NaN,NaN
1,U134616,0,NaN,446 days 08:55:01.624228,Product101,NaN,NaN,NaN,NaN
2,U130784,0,NaN,261 days 05:15:35.831008,Product101,NaN,NaN,NaN,NaN
3,U120983,1.0,32.0,76 days 18:53:27.568512,pr100340,NaN,NaN,NaN,NaN
4,U120287,143.0,485.0,45 days 05:37:40.435553,Pr100060,NaN,NaN,NaN,NaN


# OS le plus utilisé par utilisateur

In [89]:
OS_counts = df_VisitorLogs.groupby(['UserID', 'OS']).size().reset_index(name='count')
OS_max = OS_counts.loc[OS_counts.groupby('UserID')['count'].idxmax()]
OS_max.columns=['UserID','Most_Active_OS','count']
updated_values = df_Marketing_Model[['UserID']].merge(OS_max[['UserID', 'Most_Active_OS']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,239 days 05:33:17.252592,Product101,Windows,NaN,NaN,NaN
1,U134616,0,NaN,446 days 08:55:01.624228,Product101,Windows,NaN,NaN,NaN
2,U130784,0,NaN,261 days 05:15:35.831008,Product101,Chrome OS,NaN,NaN,NaN
3,U120983,1.0,32.0,76 days 18:53:27.568512,pr100340,Windows,NaN,NaN,NaN
4,U120287,143.0,485.0,45 days 05:37:40.435553,Pr100060,Windows,NaN,NaN,NaN


## Produit vu le plus récemment 

In [90]:
df_VisitorLogs.sort_values(by=['UserID','VisitDateTime'], ascending=[True,False], inplace=True)
last_product=df_VisitorLogs.groupby('UserID').first().reset_index()
last_product = last_product.rename(columns={'ProductID': 'Recently_Viewed_Product'})
updated_values=df_Marketing_Model[['UserID']].merge(last_product[['UserID','Recently_Viewed_Product']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model['Recently_Viewed_Product']=df_Marketing_Model['Recently_Viewed_Product'].fillna('Product101')
df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,239 days 05:33:17.252592,Product101,Windows,Pr100070,NaN,NaN
1,U134616,0,NaN,446 days 08:55:01.624228,Product101,Windows,Pr100495,NaN,NaN
2,U130784,0,NaN,261 days 05:15:35.831008,Product101,Chrome OS,Pr102323,NaN,NaN
3,U120983,1.0,32.0,76 days 18:53:27.568512,pr100340,Windows,pr100340,NaN,NaN
4,U120287,143.0,485.0,45 days 05:37:40.435553,Pr100060,Windows,Pr100152,NaN,NaN


## Nombre de Page généres les 7 derniers jours

In [91]:
page_load_counts=df_7jours.groupby(by='UserID')['Activity'].value_counts().loc[:, 'pageload'].reset_index(name='Pageloads_last_7_days')
updated_values = df_Marketing_Model[['UserID']].merge(page_load_counts[['UserID', 'Pageloads_last_7_days']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,239 days 05:33:17.252592,Product101,Windows,Pr100070,NaN,NaN
1,U134616,0,NaN,446 days 08:55:01.624228,Product101,Windows,Pr100495,NaN,NaN
2,U130784,0,NaN,261 days 05:15:35.831008,Product101,Chrome OS,Pr102323,NaN,NaN
3,U120983,1.0,32.0,76 days 18:53:27.568512,pr100340,Windows,pr100340,NaN,NaN
4,U120287,143.0,485.0,45 days 05:37:40.435553,Pr100060,Windows,Pr100152,4.0,NaN


## Nombre de clics des 7 derniers jours

In [92]:
page_load_counts=df_7jours.groupby(by='UserID')['Activity'].value_counts().loc[:, 'click'].reset_index(name='Clicks_last_7_days')
updated_values = df_Marketing_Model[['UserID']].merge(page_load_counts[['UserID', 'Clicks_last_7_days']], on='UserID', how='left')
df_Marketing_Model.update(updated_values)
df_Marketing_Model['Clicks_last_7_days']=df_Marketing_Model['Clicks_last_7_days'].fillna(0)
df_Marketing_Model.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U132443,0,NaN,239 days 05:33:17.252592,Product101,Windows,Pr100070,NaN,0
1,U134616,0,NaN,446 days 08:55:01.624228,Product101,Windows,Pr100495,NaN,0
2,U130784,0,NaN,261 days 05:15:35.831008,Product101,Chrome OS,Pr102323,NaN,0
3,U120983,1.0,32.0,76 days 18:53:27.568512,pr100340,Windows,pr100340,NaN,0
4,U120287,143.0,485.0,45 days 05:37:40.435553,Pr100060,Windows,Pr100152,4.0,103.0
